# flygym dev-v1.0.0 tutorial

In this tutorial, we will learn the basics of using the new API introduced in flygym v1.0.

The major changes introduced by this update are:
- refactoring: the `NeuroMechFly` has been split into `Fly`, `Camera`, and `Simulation` (or `SingleFlySimulation`).
- new features: multi-fly simulation, multi-camera rendering, etc.
- new examples: path integration, integrating connectome constrained model for vision, head stabilization, etc.
- bug fixes: e.g., fixed issue with camera not rotating according to the fly's orientation.

# Installation
To install the new version of flygym, execute the commands below in a terminal:

```sh
git clone https://github.com/NeLy-EPFL/flygym.git flygym-v1
cd flygym-v1
git checkout dev-v1.0.0
conda create -y -n flygym-v1 python=3.11
conda activate flygym-v1
pip install -e ".[dev]"
```

The commands above basically
- clone the `flygym` repository to a new directory named `flygym-v1` (to avoid conflict with the presumably existing `flygym` directory)
- change the working directory to `flygym-v1`
- switch to the `dev-v1.0.0` branch
- create a new virtual environment named `flygym-v1` (to avoid conflict with the presumably existing `flygym` environment)
- activate the `flygym-v1` environment (do this every time before you use the new `flygym` version)
- install the new version of the `flygym` package

In [1]:
from pathlib import Path
import pickle
import numpy as np
from tqdm import trange

save_dir = Path("outputs")
save_dir.mkdir(exist_ok=True)

To get started, let's do a simply kinematic reply (same as what we did in [week 1](https://github.com/NeLy-EPFL/cobar-exercises/blob/main/week1/gym_basics_and_kinematic_replay.ipynb)) with the new API. We start by loading the joint angles data:

In [2]:
# note that flygym.mujoco.<module> has been renamed to flygym.<module>
# (e.g., flygym.mujoco.preprogrammed -> flygym.preprogrammed)
from flygym.preprogrammed import all_leg_dofs
from flygym.util import get_data_path

run_time = 0.5
timestep = 1e-4
actuated_joints = all_leg_dofs

data_path = get_data_path("flygym", "data")
with open(data_path / "behavior" / "210902_pr_fly1.pkl", "rb") as f:
    data = pickle.load(f)

target_num_steps = int(run_time / timestep)
data_block = np.zeros((len(actuated_joints), target_num_steps))
input_t = np.arange(len(data["joint_LFCoxa"])) * data["meta"]["timestep"]
output_t = np.arange(target_num_steps) * timestep

for i, joint in enumerate(actuated_joints):
    data_block[i, :] = np.interp(output_t, input_t, data[joint])

# Single fly simulation
Previously, all the attributes and methods of the simulation were included in the `NeuroMechFly` class. To improve cohesion and modularity, it has been restructured into 3 classes: `Fly`, `Camera`, and `SingleFlySimulation`. Let's import them instead:

In [3]:
from flygym import Fly, Camera, SingleFlySimulation

In [4]:
# most of the parameters previously passed to Parameters are now passed to Fly.
# check the documentation for more details.
fly = Fly(
    spawn_pos=(0, 0, 0.5),
    actuated_joints=actuated_joints,
)

cam = Camera(
    fly=fly,
    camera_id="Animat/camera_left",
)

sim = SingleFlySimulation(
    fly=fly,
    cameras=[cam],
)
obs, info = sim.reset()

for i in trange(target_num_steps):
    joint_pos = data_block[:, i]
    action = {"joints": joint_pos}
    obs, reward, terminated, truncated, info = sim.step(action)
    sim.render()

# note that save_video is a method of Camera instead of Simulation
cam.save_video(save_dir / "single_fly.mp4")

100%|██████████| 5000/5000 [00:02<00:00, 1953.69it/s]


In [5]:
from IPython.display import Video

Video(str(save_dir / "single_fly.mp4"))

# Multi-camera rendering
It is also possible to pass multiple cameras to `SingleFlySimulation`:

In [6]:
import imageio

fly = Fly(
    spawn_pos=(0, 0, 0.5),
    actuated_joints=actuated_joints,
)

cameras = [
    Camera(fly=fly, camera_id=f"Animat/camera_{side}", window_size=(256, 256))
    for side in ["left", "right", "top", "front"]
]

sim = SingleFlySimulation(
    fly=fly,
    cameras=cameras,
)
obs, info = sim.reset()

with imageio.get_writer(save_dir / "multiview.mp4", fps=cameras[0].fps) as writer:
    for i in trange(target_num_steps):
        joint_pos = data_block[:, i]
        action = {"joints": joint_pos}
        obs, reward, terminated, truncated, info = sim.step(action)
        images = sim.render()

        if all(i is not None for i in images):
            frame = np.concatenate(images, axis=1)
            writer.append_data(frame)

100%|██████████| 5000/5000 [00:05<00:00, 955.16it/s] 


In [7]:
Video(str(save_dir / "multiview.mp4"))

# Multi-fly simulation
To run multi-fly simulations, use the `Simulation` class and set the `flies` parameter to a list of `Fly` objects. To identify each fly, let's name them "0" and "1". As there are multiple flies, the `action` becomes a nested dictionary indexed by the names of the flies at the top level.

In fact, `SingleFlySimulation` is just a wrapper around `Simulation`. It is provided for convenence so that `action` does not need to be indexed by the fly's name if there is only one fly.

In [8]:
from flygym import Simulation

fly0 = Fly(name="0", spawn_pos=(0, 0, 0.5))
fly1 = Fly(name="1", spawn_pos=(3, 0, 0.5))

cam = Camera(fly=fly1, camera_id="Animat/camera_right")

sim = Simulation(
    flies=[fly0, fly1],
    cameras=[cam],
)
obs, info = sim.reset()

for i in trange(target_num_steps):
    joint_pos = data_block[:, i]
    action = {
        "0": {"joints": joint_pos},
        "1": {"joints": joint_pos},
    }
    obs, reward, terminated, truncated, info = sim.step(action)
    sim.render()

cam.save_video(save_dir / "two_flies.mp4")

100%|██████████| 5000/5000 [00:04<00:00, 1001.12it/s]


Simularly, the observations dictionary is now first indexed by the flies' names, then by the variable names.

In [21]:
print(list(obs.keys()))
print(list(obs["0"].keys()))

['0', '1']
['joints', 'fly', 'contact_forces', 'end_effectors', 'fly_orientation']


Refer to [chasing.ipynb](chasing.ipynb) for an example of a fly chasing another fly.